# Tumor Mutation Classification

In this project, we will analyze the dataset which contains tumor gene mutations and their risk category, which corresponds to their risk of malignancy in the human condition. The dataset was hand-labeled and released by the team of clinical pathologists at Memorial Sloan Kettering in 2018. Our objective of this project is to fit the dataset into our machine learning models to predict the risk category while accounting for highly unbalanced classes. Several methods for text feature generation will be explored and the resulting features will be reduced using principle component analysis (PCA). We will then use the synthetic minority over-sampling technique (SMOTE) to resample the dataset to make the numbers of categories more even. The last step is to compare the machine learning methods.

In [1]:
import pandas as pd
import boto3
import io

import re
import spacy

from collections import Counter

from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from imblearn.over_sampling import SMOTE

from sklearn import linear_model, decomposition, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
import xgboost
from xgboost import XGBClassifier


#from spacy.lemmatizer import Lemmatizer
#lemmatizer = Lemmatizer()
#nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
nlp = spacy.blank('en')
#nlp.add_pipe(lemmatizer)

# Preprocessing

In this section we will load in all our data and format them to the appropriate data types. The text also needs to be cleaned of all non-legitimate words, such as figure references and parentheses. This will be accomplished using regular expressions.

In [3]:
# Set up packages for loading in data
client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API

In [4]:
# Load in training data labels
obj = client.get_object(Bucket='thinkful-capstone', Key='training_variants')
stream = io.BytesIO(obj['Body'].read())
training_variants = pd.read_csv(stream)

In [5]:
print(training_variants.head())
print(training_variants.shape)

   ID    Gene             Variation  Class
0   0  FAM58A  Truncating Mutations      1
1   1     CBL                 W802*      2
2   2     CBL                 Q249E      2
3   3     CBL                 N454D      3
4   4     CBL                 L399V      4
(3321, 4)


In [6]:
# Load in training data text articles
obj = client.get_object(Bucket="thinkful-capstone",Key="training_text")
raw_training_text = obj["Body"].read()
training_text = raw_training_text.decode('utf-8')
print(training_text[:10000])

ID||Text
0||Cyclin-dependent kinases (CDKs) regulate a variety of fundamental cellular processes. CDK10 stands out as one of the last orphan CDKs for which no activating cyclin has been identified and no kinase activity revealed. Previous work has shown that CDK10 silencing increases ETS2 (v-ets erythroblastosis virus E26 oncogene homolog 2)-driven activation of the MAPK pathway, which confers tamoxifen resistance to breast cancer cells. The precise mechanisms by which CDK10 modulates ETS2 activity, and more generally the functions of CDK10, remain elusive. Here we demonstrate that CDK10 is a cyclin-dependent kinase by identifying cyclin M as an activating cyclin. Cyclin M, an orphan cyclin, is the product of FAM58A, whose mutations cause STAR syndrome, a human developmental anomaly whose features include toe syndactyly, telecanthus, and anogenital and renal malformations. We show that STAR syndrome-associated cyclin M mutants are unable to interact with CDK10. Cyclin M silencing pheno

In [7]:
# Eliminate references and abbreviations within parentheses
print(len(training_text))
training_text = re.sub(' \(Fig \d+.+?\)', '', training_text)
training_text = re.sub(' \(Fig\. \d+.+?\)', '', training_text)
training_text = re.sub(' \(\d.*?\)', '', training_text)
training_text = re.sub(' \([A-Z]\)', '', training_text)
print(len(training_text))
print(training_text[:4000])

211296707
205598350
ID||Text
0||Cyclin-dependent kinases (CDKs) regulate a variety of fundamental cellular processes. CDK10 stands out as one of the last orphan CDKs for which no activating cyclin has been identified and no kinase activity revealed. Previous work has shown that CDK10 silencing increases ETS2 (v-ets erythroblastosis virus E26 oncogene homolog 2)-driven activation of the MAPK pathway, which confers tamoxifen resistance to breast cancer cells. The precise mechanisms by which CDK10 modulates ETS2 activity, and more generally the functions of CDK10, remain elusive. Here we demonstrate that CDK10 is a cyclin-dependent kinase by identifying cyclin M as an activating cyclin. Cyclin M, an orphan cyclin, is the product of FAM58A, whose mutations cause STAR syndrome, a human developmental anomaly whose features include toe syndactyly, telecanthus, and anogenital and renal malformations. We show that STAR syndrome-associated cyclin M mutants are unable to interact with CDK10. Cycl

In [8]:
# Split text file into list of documents
training_list = training_text.split('||')
training_list = training_list[2:]
print(len(training_list))

3321


In [9]:
# Load training text into dataframe
texts_df = pd.DataFrame(training_list, columns = ['text'])

# Merge text dataframe with labels dataframe
train = pd.concat([training_variants, texts_df], axis=1)
print(train.head())
print(train.shape)

   ID    Gene             Variation  Class  \
0   0  FAM58A  Truncating Mutations      1   
1   1     CBL                 W802*      2   
2   2     CBL                 Q249E      2   
3   3     CBL                 N454D      3   
4   4     CBL                 L399V      4   

                                                text  
0  Cyclin-dependent kinases (CDKs) regulate a var...  
1   Abstract Background  Non-small cell lung canc...  
2   Abstract Background  Non-small cell lung canc...  
3  Recent evidence has demonstrated that acquired...  
4  Oncogenic mutations in the monomeric Casitas B...  
(3321, 5)


In [10]:
# Temporary cell to reduce data size
#train = train[:400]
#print(train.shape)

Now that the data is loaded into a dataframe, let's do some preliminary data exploration.

In [11]:
print(train['Class'].value_counts())

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64


We have 3321 total datapoints to work with, and it looks like we are dealing with significant class imbalance. Class 7 has 953 datapoints, while Class 8 has only 19. We will have to address this class imbalance with our experiment design. Additionally, the labels have been anonymized, which means we cannot draw any insight about what these classes might signify. 

# Experiment Design

The prevalence of class imbalance has serious implications for our analysis. First and foremost, we must establish our scoring metric. The purpose here is to use the relevant clinical texts to predict the mutation category for each gene/mutation pair. While we want the predictions to be as accurate as possible, simple classification accuracy is not a representative way to judge models that are built on class imbalance, as they may achieve high accuracy by simply predicting the most common class every time. <br>

Given that we are working with a multi-label classifier, the most appropriate scoring metric is Cohen's Kappa coefficient (K). This statistic measures inter-rater agreement for categorical items. Contrary to conventional classification accuracy, the K statistic accounts for the possibility of the agreement occurring by chance. <br>

The Kappa statistic varies from 0 to 1, where 0 = agreement equivalent to chance, and 1 = perfect agreement. We will be choosing models with Kappa statistics closer to 1. We will also look at the precision and recall via the F1 score. Though these are not optimized for multi-label classification, they will be interesting to consider. <br>

We will also try oversampling the lesser-represented categories and apply our machine learning models on the oversampled datasets, judging by their Kappa statistic. Oversampling can be achieved by generating duplicate datapoints or by generating new synthetic datapoints via SMOTE, the Synthetic Minority Oversampling Technique. <br>

I will use various methods of feature generation including classic NLP techniques such as bag-of-words, tf-idf, and n-grams. These methods of feature generation will be applied to both the original and oversampled datasets. They will then be subjected to various machine learning models. Decision trees are known to perform well on unbalanced datasets, so this model may prevail on the original data. However, Naive Bayes is known to perform well on natural langauge, so once the dataset is oversampled it is possible that Naive Bayes will perform the best.

# Data Cleaning

The data is relatively clean already, and contains no NaN values. It needs to be tokenized so it can be processed into readable pieces of data. We will use spaCy to tokenize the data and create a new column with a list of the tokens for each row. Furthermore, we will convert all tokens that are not stop words or punctuation to lemmas to reduce the noise from unnecessary words.

In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'to', 'were', 'haven', 'through', 'herself', 'an', 'of', "weren't", "hasn't", "you're", 'aren', 'during', 'between', 'who', "hadn't", 'this', 'too', 'ain', "you'll", 'she', 'will', 'up', 'only', "should've", 'mightn', 'our', 'ourselves', 'ours', "you'd", "don't", 'll', 'hasn', 'should', 'y', 'hers', 'down', 'below', "she's", 'while', 'again', 'didn', 'all', 'am', 'hadn', "needn't", 'than', 'the', 'ma', 'under', 'if', 'from', 'each', 'yourselves', 'once', "shan't", 'more', 've', 'wasn', 'then', 'can', 'won', 'there', 'how', 'isn', 'o', "won't", 'does', "it's", "isn't", 'because', "wasn't", "couldn't", 'when', "wouldn't", 'before', "that'll", 'we', 'these', 'had', 'you', 'their', 'his', 'here', "mustn't", 'wouldn', 'about', 'whom', 'off', 'other', 'being', 'for', 'he', 'is', 'just', 'i', 'so', 'both', 'don', 'they', 'on', 'own', 'not', "mightn't", 'where', 'same'

In [13]:
print('beginning lemmatization')
train['spacy_tokens'] = train['text'].apply(lambda x: nlp(x))

def lemmatize(x):
    intermediate_lemmas = [token.lemma_.lower() for token in x
            if not token.is_punct]
    return [lemma for lemma in intermediate_lemmas
           if lemma not in stop_words
           and lemma != "-PRON-"
           and lemma != " "
           ]

train['lemmas'] = train['spacy_tokens'].apply(lambda x: lemmatize(x))
print(train.head())

beginning lemmatization


KeyboardInterrupt: 

idea: export lemmatized to file to speed up future runs
Convert into a list of strings to feed to tf-idf vectorizer

In [ ]:
# Establishing our test datasets

X = train['lemmas']
Y = train['Class']

X_lemma_documents = [
    ' '.join([str(word) for word in text])
    for text in X.values.tolist()
]

In [ ]:
common_X_lemma_documents = []

# creates a list of the N most common words per document to reduce input to vectorizer
for document in X_lemma_documents: 
    N = 10000
    words = re.findall(r'\w+', document)
    lower_words = [word.lower() for word in words]
    word_counts = Counter(lower_words).most_common(N)

    common_words = [word_tuple[0] for word_tuple in word_counts]
    common_X_lemma_documents.append(common_words)
    
print(len(common_X_lemma_documents))

# Feature Generation

https://docs.microsoft.com/en-us/machine-learning-server/python-reference/microsoftml/featurize-text 

from microsoftml import rx_logistic_regression, featurize_text, rx_predict
from microsoftml.entrypoints._stopwordsremover_predefined import predefined

makes it incredibly easy but won't install

what about tSNE dimensionality reduction before tf-idf vectorization? they aren't technically features yet... (use PCA first) <br>
https://lvdmaaten.github.io/tsne/

# Creating n-grams

Beyond using singular words or lemmas as features for classification, we can use groupings of words that appear together, as they may convey more meaning than each word isolated by itself. We will create features for bigrams and trigrams, as any groupings larger than 3 words will likely be too specific and may create unnecessary noise.

In [ ]:
def ngrams(input_text, n):
  input_text = input_text.split(' ')
  output = []
  for i in range(len(input_text)-n+1):
    output.append(input_text[i:i+n])
  return output

bigrams = []

for document in X_lemma_documents:
    document_bigrams = ngrams(document, 2)
    #for bigram in document_bigrams:
        #bigrams.append(bigram)
    
#print(bigrams[:1])
print(document_bigrams)
# currently returns as nested, do we need to flatten into one list of lists?

In [ ]:
joined_bigrams = []

for bigram in document_bigrams:
    joined_bigram = bigram[0] + ' ' + bigram[1]
    joined_bigrams.append(joined_bigram)

print(joined_bigrams[:5])
bigram_counts = Counter(joined_bigrams).most_common(10000)
print(bigram_counts[:10])

common_bigrams = [bigram[0] for bigram in bigram_counts]
print(common_bigrams[:3])


#common_X_lemma_documents.append(common_words)

# TF-IDF Vectorization

# Change eta to learning rate

In [ ]:
pipe = Pipeline(steps=[
    ('feat_gen', TfidfVectorizer()),
    ('reduce_dim', decomposition.TruncatedSVD()),
    ('oversample', SMOTE(k_neighbors = 1)),
    # increase k neighbors to default
    # add configuration without oversampling/dim red
    ('machine', GaussianNB())
    # why gaussian?
])

# Feature generation paramters
maxdf = [0.25, 0.5, 0.75]
mindf = [0.01, 0.05, 0.1]

# Dimensionality reduction parameters
n_components = [500]
# why dim red? why 500?
# if time, show scree plot

# Oversampling parameters
smote_kind = ['regular','borderline1']

# Machine learning model parameters 
rf_max_depth = [10, 20, 30]
rf_max_features = [10, 20, 30]
rf_min_samples_split = [10, 20, 30]
# look into parameter choices for RF for high dimensionality. consider higher values
rf_n_estimators = [10, 20]
xg_booster = ['gbtree','gblinear']
xg_eta = [0.01, 0.1, 0.3]
xg_max_depth = [5, 10]
xg_subsample = [0.5, 1]



param_grid = [
    # Tfidf permutations first, running through all machine learning models
    {
        'feat_gen': [TfidfVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [GaussianNB()] 
    },
    {
        'feat_gen': [TfidfVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [RandomForestClassifier(random_state = 1)],
        'machine__max_depth': rf_max_depth,
        'machine__max_features': rf_max_features,
        'machine__min_samples_split': rf_min_samples_split,
        'machine__n_estimators': rf_n_estimators
    },
    {
        'feat_gen': [TfidfVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [XGBClassifier()],
        'machine__booster': xg_booster,
#        'machine__eta': xg_eta,
        'machine__max_depth': xg_max_depth,
        'machine__subsample': xg_subsample
    },
    # Bag of words permutations next, running through all machine learning models
    {
        'feat_gen': [CountVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [GaussianNB()]
    },
    {
        'feat_gen': [CountVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [RandomForestClassifier(random_state = 1)],
        'machine__max_depth': rf_max_depth,
        'machine__max_features': rf_max_features,
        'machine__min_samples_split': rf_min_samples_split,
        'machine__n_estimators': rf_n_estimators
    },
    {
        'feat_gen': [CountVectorizer()],
        'feat_gen__max_df': maxdf,
        'feat_gen__min_df': mindf,
        'reduce_dim__n_components': n_components,
        'oversample__kind': smote_kind,
        'machine': [XGBClassifier()],
        'machine__booster': xg_booster,
        'machine__eta': xg_eta,
        'machine__max_depth': xg_max_depth,
        'machine__subsample': xg_subsample
    }

]

grid = GridSearchCV(pipe, cv=3, n_jobs=4, param_grid=param_grid, verbose=2, scoring = 'log_loss')

In [ ]:
grid.fit(X_lemma_documents, Y)
print(f'best params:\n {grid.best_params_}')

In [ ]:
import csv
import numpy as np
from operator import itemgetter
import datetime

def export_gridsearch_to_csv(gs_clf, export_file):
    with open(export_file, 'w') as outfile:
        csvwriter = csv.writer(outfile, delimiter=',')

        # Create the header using the parameter names 
        header = ["mean","std", "params"]

        csvwriter.writerow(header)
        
        sorted_by_score = sorted(gs_clf.grid_scores_, key = itemgetter(1), reverse=True)
        
        for config in sorted_by_score:
            # Get mean and standard deviation
            mean = np.abs(config[1])
            std = np.std(config[2])
            row = [mean,std, str(config[0])]

            csvwriter.writerow(row)
            
filename = datetime.datetime.utcnow().strftime('gridresults_%Y%m%d_%H:%M.csv')       
export_gridsearch_to_csv(grid, filename)

In [ ]:
import datetime
filename = datetime.datetime.utcnow().strftime('gridresults_%Y%m%d_%H%M.csv')

# Extra configurations

# Original custom tfidf vectorizer function

In [ ]:
def tfidf_vectorizer(X,Y):
    vectorizer = TfidfVectorizer(max_df=0.5,
                                 min_df=2,
                                 stop_words='english', 
                                 lowercase=True,
                                 norm=u'l2',
                                 smooth_idf=True,
                                )

    sparse_tfidf_matrix=vectorizer.fit_transform(X_strings)
    print(f'Number of features: {sparse_tfidf_matrix.get_shape()[1]}')
    
    # Densify matrix so we can convert it to a conventional dataframe to extract X/Y
    dense_tfidf_matrix = sparse_tfidf_matrix.todense()
    df_tfidf = pd.DataFrame(dense_tfidf_matrix)
    df_tfidf['lemma_tokens'] = X
    df_tfidf['class'] = Y
    
    return df_tfidf

tfidf_unigram = tfidf_vectorizer(X_strings,Y)

In [ ]:
tfidf_bigram = tfidf_vectorizer(bigrams, Y)

In [ ]:
print(tfidf_df.head())

# Machine Learning Methods

Here, we will attempt to classify the training data using several different machine learning classifiers.

# Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(tX_train, tY_train)
test_pred = gnb.predict(tX_test)
print(f'Testing Accuracy: {accuracy_score(tY_test, test_pred)}')
print(f'Cross Val Score: {cross_val_score(gnb, tX_train, tY_train, cv=3).mean()}')
print(pd.crosstab(tY_test, test_pred))

In [ ]:
tfidf = tfidf_vectorizer(
                            stop_words='english', 
                            lowercase=True,
                            norm=u'l2',
                            smooth_idf=True
                        )
nb = GaussianNB()

pipe = Pipeline(steps=[('tfidf', tfidf), ('nb', nb)])

tfidf_maxdf = [0.25, 0.5, 0.75]
tfidf_mindf = [2, 5, 20]

param_grid = [
    {
        'tfidf__max_df': tfidf_maxdf,
        'tfidf__min_df': tfidf_mindf,
    }
]

In [ ]:
grid = GridSearchCV(pipe, cv=5, n_jobs=1, param_grid=param_grid)
grid.fit(X_train, Y_train)
#print(f'best params:\n {grid.best_params_}')

In [ ]:
from sklearn import linear_model, decomposition, datasets

pipe = Pipeline([
    ('feat_gen', tfidf_vectorizer()),
    ('reduce_dim', decomposition.PCA())
    ('classify', GaussianNB())
])

tfidf_maxdf = [0.25, 0.5, 0.75]
tfidf_mindf = [2, 5, 20]

param_grid_1 = [
    {
        'feat_gen': [tfidf_vectorizer(                            
                                        stop_words='english', 
                                        lowercase=True,
                                        norm=u'l2',
                                        smooth_idf=True
                                    )],
        'feat_gen__max_df': tfidf_maxdf,
        'feat_gen__min_df': tfidf_mindf,
        'classify__C': C_OPTIONS
    },
    {
        'feat_gen': [SelectKBest(chi2)],
        'feat_gen__k': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS
    },
]

param_grid_2 = dict(feat_gen=[tfidf_vectorizer(), count_vectorizer()],
                    reduce_dim = [PCA(), tSNE()]
                    clf = [GaussianNB()])


grid = GridSearchCV(pipe, cv=3, n_jobs=1, param_grid=param_grid)
grid.fit(X, Y)

##### needs to use a real class not a custom function
##### input must be X list of strings (each string corresponds to a single document) and Y of same size

# SMOTE example

In [ ]:
sm = SMOTE(random_state=42)
X_res, Y_res = sm.fit_sample(X, Y)
print('Resampled dataset shape {}'.format(Counter(y_res)))
print('The number of transactions after resampling : ' + str(len(X_res)))
print(X_res.value_count())